In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import os
import json

import websockets
import asyncio
import nest_asyncio
nest_asyncio.apply() # to run streams in jupyter notebook (where an event loop is already active)
import threading

from alpaca.data import \
CryptoHistoricalDataClient, StockHistoricalDataClient, OptionHistoricalDataClient, \
CryptoBarsRequest,StockBarsRequest, StockLatestQuoteRequest, CryptoLatestQuoteRequest, TimeFrame

from alpaca.trading import \
TradingClient, GetAssetsRequest, AssetClass, GetOrdersRequest, QueryOrderStatus, \
LimitOrderRequest, MarketOrderRequest, OrderSide, TimeInForce, TradingStream

from alpaca.data.live import CryptoDataStream, OptionDataStream, StockDataStream

KEY = os.getenv('ALPACA_KEY_live')
SECRET = os.getenv('ALPACA_SECRET_live')

In [2]:
async def alpaca_crypto_stream():
    # Use the live URL. For sandbox testing, switch to:
    # "wss://stream.data.sandbox.alpaca.markets/v1beta3/crypto/us"
    url = "wss://stream.data.alpaca.markets/v1beta3/crypto/us"
    
    async with websockets.connect(url) as ws:
        # Connection messages from the server.
        response = await ws.recv()
        print("Connected:", response)
        
        # --- Authenticate ---
        auth_msg = {
            "action": "auth",
            "key": KEY,       # Replace with your Alpaca API key
            "secret": SECRET  # Replace with your Alpaca API secret
        }
        await ws.send(json.dumps(auth_msg))
        auth_response = await ws.recv()
        print("Auth Response:", auth_response)
        
        # --- Subscribe to BTC/USD trades ---
        sub_msg = {
            "action": "subscribe",
            "trades": ["BTC/USD"]
            # For quotes or bars, you could add:
            # "quotes": ["BTC/USD"], "bars": ["BTC/USD"], etc.
        }
        await ws.send(json.dumps(sub_msg))
        sub_response = await ws.recv()
        print("Subscription Response:", sub_response)
        
        # --- Start receiving streaming data ---
        print("Starting to receive streaming data...")
        while True:
            try:
                message = await ws.recv()
                data = json.loads(message)
                print("Received Data:")
                print(json.dumps(data, indent=2))
            except websockets.ConnectionClosed:
                print("Connection closed")
                break
            except Exception as e:
                print("Error:", e)



In [ ]:
await alpaca_crypto_stream()

Connected: [{"T":"success","msg":"connected"}]
Auth Response: [{"T":"success","msg":"authenticated"}]
Subscription Response: [{"T":"subscription","trades":["BTC/USD"],"quotes":[],"orderbooks":[],"bars":[],"updatedBars":[],"dailyBars":[]}]
Starting to receive streaming data...
Received Data:
[
  {
    "T": "t",
    "S": "BTC/USD",
    "p": 107044.543,
    "s": 4.5106e-05,
    "t": "2025-05-25T16:06:09.935812121Z",
    "i": 8055144525740527097,
    "tks": "B"
  }
]
Received Data:
[
  {
    "T": "t",
    "S": "BTC/USD",
    "p": 107044.543,
    "s": 0.000117277,
    "t": "2025-05-25T16:06:09.936240481Z",
    "i": 2552412708424732499,
    "tks": "B"
  }
]
Received Data:
[
  {
    "T": "t",
    "S": "BTC/USD",
    "p": 106992.6,
    "s": 0.000225643,
    "t": "2025-05-25T16:06:22.344434596Z",
    "i": 5136862475455448006,
    "tks": "B"
  }
]
Received Data:
[
  {
    "T": "t",
    "S": "BTC/USD",
    "p": 106964.411,
    "s": 9.9309e-05,
    "t": "2025-05-25T16:06:22.873984692Z",
    "i": 3